In [1]:
import pandas as pd
pd.options.display.max_columns = 150
pd.options.display.max_rows = 150

df = pd.read_csv("df_clean.csv")

df.head()

,album_type,track_duration,nombre_artist,track_release_month,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,time_signature,pays_du_producteur,popularity
0,album,30000,1,December,0.3210,0.952,6.0,-16.922,1.0,0.2100,0.7050,0.978,0.3400,0.03230,101.659,4.0,AU,0.0
1,album,30000,1,January,0.2750,0.912,3.0,-23.424,1.0,0.0704,0.9670,0.916,0.1990,0.00000,37.771,4.0,US,35.0
2,album,30000,1,March,0.1760,0.999,2.0,-25.388,1.0,0.0786,0.0677,0.662,0.7620,0.00001,98.109,4.0,ZZ,59.0
3,album,30000,1,December,0.0837,0.208,7.0,-20.947,1.0,0.0422,0.7000,0.880,0.0606,0.06500,168.334,5.0,GB,35.0
4,album,30000,1,December,0.2140,0.236,7.0,-20.772,1.0,0.0448,0.6780,0.878,0.0637,0.03920,79.725,4.0,GB,38.0


In [3]:
df['key']=df[('key')].astype(object)
from sklearn.model_selection import train_test_split

X=df.drop(['popularity'],axis=1)
y=df['popularity']
X_train,X_test,y_train,y_test = train_test_split(X, y,shuffle=True, test_size=0.2,random_state=42,stratify=X['pays_du_producteur'])



In [4]:
numeric_features = list(X.select_dtypes(include=['int', 'float']).columns)
categorial_features=list(X.select_dtypes( exclude=['int', 'float']).columns)
print(numeric_features)
print(categorial_features)

['track_duration', 'nombre_artist', 'danceability', 'energy', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'time_signature']
['album_type', 'track_release_month', 'key', 'pays_du_producteur']


In [5]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler,PolynomialFeatures
from sklearn.decomposition import PCA

numeric_transformer = Pipeline([
        ('standard', StandardScaler()),  # Centrer réduire
        ('pca', PCA(n_components=8)),
         ('polyanomie',PolynomialFeatures()) # après standardisation
        ])

from sklearn.preprocessing import OneHotEncoder
# categorical_transformer = OneHotEncoder(sparse=True)

categorical_transformer = Pipeline([
         ('onehot',OneHotEncoder(drop='if_binary')), 
         ('polyanomie',PolynomialFeatures()) # après standardisation
        ])



# ############### IV.d Combinaison ##############
from sklearn.compose import ColumnTransformer

preprocessor = ColumnTransformer(
    transformers=[
        ('num', numeric_transformer, numeric_features),
        ('cat', categorical_transformer, categorial_features)
    ],)
    # remainder="passthrough" )

In [7]:

from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
lr_model = LinearRegression()
pipe_rfr = Pipeline([
     ('preprocessor', preprocessor),
     ('decisiontree', lr_model)
])


In [8]:
import mlflow


mlflow.set_experiment("linearregressor_models")
mlflow.sklearn.autolog()
# premier run
with mlflow.start_run(run_name='reg_linear_regressor'):
  
  pipe_rfr.fit(X_train, y_train)

2023/03/02 16:20:53 INFO mlflow.tracking.fluent: Experiment with name 'linearregressor_models' does not exist. Creating a new experiment.
2023/03/02 16:20:53 WARNING mlflow.utils.autologging_utils: You are using an unsupported version of sklearn. If you encounter errors during autologging, try upgrading / downgrading sklearn to a supported version, or try upgrading MLflow.
2023/03/02 16:20:53 WARNING mlflow.utils: Truncated the value of the key `steps`. Truncated value: `[('preprocessor', ColumnTransformer(transformers=[('num',
                                 Pipeline(steps=[('standard', StandardScaler()),
                                                 ('pca', PCA(n_components=8)),
                                                 ('polyanomie',
                                                  PolynomialFeatures())]),
                                 ['track_duration', 'nombre_artist',
                                  'danceability', 'energy', 'loudness',...`
2023/03/02 16:20:53 WA